# 📊 02 – Factor Modeling with PCA and Regression

## Overview

This notebook implements a **multi-factor model** to understand how macroeconomic factors influence emerging market equity returns. We use:

1. **Principal Component Analysis (PCA)** to reduce dimensionality of macro factors
2. **Linear Regression** to model EM equity sensitivity to macro principal components  
3. **Factor loadings analysis** to interpret economic relationships
4. **Model performance evaluation** using R² scores

### Methodology:
- **Data Transformation**: Convert prices to log returns for stationarity
- **Standardization**: Scale macro variables for PCA
- **Dimensionality Reduction**: Extract 3 principal components explaining most variance
- **Factor Regression**: Model each EM index as function of macro PCs
- **Visualization**: Heatmaps, explained variance, and actual vs. predicted plots

## 📦 Import Required Libraries

Loading essential libraries for factor modeling and analysis:

In [ ]:
# Core data manipulation
import pandas as pd
import numpy as np
import os

# Machine learning components
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

## 📁 Data Loading & Preparation

Load the combined EM and macro dataset, then transform to log returns for stationarity.

In [ ]:
# Load the combined dataset from previous notebook
df = pd.read_csv('../data/combined_em_macro_data.csv', parse_dates=['date'], index_col='date')

print(f"📊 Dataset loaded: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"📅 Date range: {df.index.min()} to {df.index.max()}")

# Convert prices to log returns (more stationary for modeling)
log_returns = np.log(df / df.shift(1)).dropna()

print(f"📈 Log returns calculated: {log_returns.shape[0]} observations")
print(f"🧹 Data cleaning: {df.shape[0] - log_returns.shape[0]} rows dropped (missing/infinite values)")

# Display basic statistics
print(f"\n📋 Log Returns Summary:")
log_returns.describe().round(4)

## 🎯 Variable Separation

Separate the dataset into dependent variables (EM returns) and independent variables (macro factors).

In [ ]:
# Identify EM equity columns (dependent variables)
em_columns = [col for col in df.columns if col.startswith(('Brazil', 'India', 'China', 'SouthAfrica', 'Mexico', 'Indonesia'))]

# Identify macro factor columns (independent variables)  
macro_columns = [col for col in df.columns if col not in em_columns]

print(f"🌏 EM Equity Variables ({len(em_columns)}):")
for col in em_columns:
    print(f"   • {col}")

print(f"\n📈 Macro Factor Variables ({len(macro_columns)}):")
for col in macro_columns:
    print(f"   • {col}")

# Separate into Y (dependent) and X (independent) matrices
Y = log_returns[em_columns]  # EM equity returns to predict
X = log_returns[macro_columns]  # Macro factors as predictors

print(f"\n📊 Model Setup:")
print(f"   • Y matrix (EM returns): {Y.shape}")
print(f"   • X matrix (Macro factors): {X.shape}")

## 🔍 Principal Component Analysis (PCA)

Apply PCA to macro factors to reduce dimensionality while retaining most of the variance. This helps avoid multicollinearity and simplifies interpretation.

In [ ]:
# Step 1: Standardize macro factors (required for PCA)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"📏 Standardization completed:")
print(f"   • Original X shape: {X.shape}")
print(f"   • Scaled X shape: {X_scaled.shape}")
print(f"   • Mean: {X_scaled.mean():.6f}")
print(f"   • Std: {X_scaled.std():.6f}")

# Step 2: Apply PCA to extract 3 principal components
n_components = 3
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

# Step 3: Analyze explained variance
explained_var = pca.explained_variance_ratio_
cumulative_var = np.cumsum(explained_var)

print(f"\n🔍 PCA Results:")
for i in range(n_components):
    print(f"   • PC{i+1}: {explained_var[i]:.1%} variance explained")
print(f"   • Total: {cumulative_var[-1]:.1%} variance captured")

print(f"\n📊 Principal Components Matrix: {X_pca.shape}")

In [ ]:
# Visualize explained variance
plt.figure(figsize=(10, 4))

# Subplot 1: Individual explained variance
plt.subplot(1, 2, 1)
plt.bar(range(1, n_components + 1), explained_var, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('PCA: Individual Explained Variance')
plt.xlabel('Principal Component')
plt.ylabel('Variance Ratio')
plt.grid(axis='y', alpha=0.3)

# Subplot 2: Cumulative explained variance  
plt.subplot(1, 2, 2)
plt.plot(range(1, n_components + 1), cumulative_var, marker='o', linewidth=2, markersize=8, color='red')
plt.title('PCA: Cumulative Explained Variance')
plt.xlabel('Principal Component')
plt.ylabel('Cumulative Variance Ratio')
plt.grid(True, alpha=0.3)
plt.ylim(0, 1)

plt.tight_layout()
plt.show()

print(f"✅ PCA captures {cumulative_var[-1]:.1%} of macro factor variance with {n_components} components")

## 📈 Factor Regression Analysis

Fit linear regression models for each EM equity index using the principal components as explanatory variables.

In [ ]:
# Fit linear regression for each EM index against principal components
betas = {}       # Store factor loadings (sensitivities)
r2_scores = {}   # Store model fit statistics

print(f"🔄 Fitting {len(Y.columns)} regression models...\n")

for col in Y.columns:
    # Fit regression: EM_return = α + β₁×PC1 + β₂×PC2 + β₃×PC3 + ε
    model = LinearRegression().fit(X_pca, Y[col])
    
    # Store results
    betas[col] = model.coef_
    r2_scores[col] = model.score(X_pca, Y[col])
    
    print(f"✅ {col}:")
    print(f"   • R² Score: {r2_scores[col]:.3f}")
    print(f"   • Factor Loadings: [{betas[col][0]:.3f}, {betas[col][1]:.3f}, {betas[col][2]:.3f}]")

# Create DataFrame for factor loadings (betas)
beta_df = pd.DataFrame(betas, index=['PC1', 'PC2', 'PC3']).T
beta_df.index.name = 'EM Index'

print(f"\n📊 Factor Loadings Matrix:")
print(beta_df.round(3))

In [ ]:
# Visualize factor loadings as heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(beta_df, annot=True, cmap='RdBu_r', center=0, 
            fmt='.3f', cbar_kws={'label': 'Factor Loading'})
plt.title('Factor Loadings: EM Equity Sensitivity to Macro Principal Components', fontsize=14, pad=20)
plt.xlabel('Principal Component')
plt.ylabel('EM Equity Index')
plt.tight_layout()
plt.show()

# Create R² scores summary
r2_df = pd.DataFrame.from_dict(r2_scores, orient='index', columns=['R² Score'])
r2_df = r2_df.sort_values('R² Score', ascending=False)

# Plot R² scores
plt.figure(figsize=(10, 5))
bars = plt.bar(range(len(r2_df)), r2_df['R² Score'], color='skyblue', edgecolor='black', alpha=0.7)
plt.title('Model Fit (R²) by EM Equity Index', fontsize=14)
plt.xlabel('EM Equity Index')
plt.ylabel('R² Score')
plt.xticks(range(len(r2_df)), r2_df.index, rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.005,
             f'{height:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"\n📈 Model Performance Summary:")
print(f"   • Best fit: {r2_df.index[0]} (R² = {r2_df.iloc[0, 0]:.3f})")
print(f"   • Worst fit: {r2_df.index[-1]} (R² = {r2_df.iloc[-1, 0]:.3f})")
print(f"   • Average R²: {r2_df['R² Score'].mean():.3f}")

## 🎯 Model Validation & Visualization

Generate actual vs. predicted plots for each EM index and save results for the output folder.

In [ ]:
# Create output directory for plots
plot_dir = '../output/plots'
os.makedirs(plot_dir, exist_ok=True)

# Generate actual vs predicted plots for all EM indices
print(f"📊 Generating actual vs. predicted plots for {len(Y.columns)} EM indices...\n")

for col in Y.columns:
    # Fit model and generate predictions
    model = LinearRegression().fit(X_pca, Y[col])
    Y_pred = model.predict(X_pca)
    r2 = model.score(X_pca, Y[col])
    
    # Create plot
    plt.figure(figsize=(12, 5))
    
    plt.plot(Y.index, Y[col], label='Actual', linewidth=1.5, alpha=0.8)
    plt.plot(Y.index, Y_pred, label='Predicted (PCA Model)', linestyle='--', linewidth=1.5)
    
    plt.title(f'{col} Returns: Actual vs Predicted (R² = {r2:.3f})', fontsize=14)
    plt.xlabel('Date')
    plt.ylabel('Log Returns')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    # Save plot
    filename = col.replace(" ", "_").replace("/", "_") + '.png'
    plot_path = os.path.join(plot_dir, filename)
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ {col}: R² = {r2:.3f}, Plot saved to {filename}")

print(f"\n💾 All plots saved to: {plot_dir}")
print(f"🎯 Analysis complete! Factor modeling successfully applied to {len(Y.columns)} EM indices.")